In [12]:
from preparator import Preparator 
import numpy as np 
from solver import Solver 
prep = Preparator()

from preparator import Preparator 
from logger import Logger 
import pandas as pd 
from datetime import datetime
import tqdm 
#Header
# Id (int),
# Valid (int),
# Updated (str),
# Matrix (mat),
# Size (int),
# Type (str),
# Rank (int),
# Density (flt),
# Stiffness (flt),
# Other (flt),
# Test_solver (str),
# Test (flt),
# Test_state (vec),
# Test_iterations (int),
# Test_time (flt),
# Train_solver (str),
# Preprocessing (str),
# Postprocessing (str),
# Train (flt),
# Train_state (vec),
# Train_iterations_1 (int),
# Train_iterations_2 (int),
# Train_iterations_3 (int),
# Train_shots (int),
# Train_cost (flt),
# Train_time (flt),
# Baren_flag (str)



In [20]:
# Prepare source dataset 

source_name = 'data/stiffness/Stiffness_size_{size}_stiffs_{stiffs}_stepsize_{stepsize}_shots_{shots}.csv'

Config = {
    'size' : 10,
    'stiffs' : [1e0, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6],
    'stiff_count' : 10, 
    'shots' : 1000, 
    'stepsize' : 0.25, 
    'accepterr': [0.001,0.01,0.01], 
    'iterations_limit' : 1000, 
    'baren_err' : 0.001 / 10,
    'baren_threshold': 10
}

template = pd.read_csv('data/template.csv')

log = Logger (log_template = template, log_path = source_name.format(size = Config['size'], stiffs = Config['stiffs'], stepsize = Config['stepsize'], shots = Config['shots']))
exists = log.create_logfile()


log_file, id = log._load_logfile()


Logfile data/stiffness/Stiffness_size_10_stiffs_[1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0, 1000000.0]_stepsize_0.25_shots_1000.csv already exists


In [14]:

# for stiff in Config['stiffs']: 
#     for i in tqdm.tqdm(range(Config['stiff_count']), desc = f"Generating matrices with stiffness {stiff}"):
        
#         id += 1
#         matrix = prep.stiffnessPreparator(size = Config['size'], stiffness = stiff)
#         current_datetime = datetime.now()
#         data = { 
#             'Id (int)' : id, 
#             'Valid (int)' : 1, 
#             'Updated (str)' : current_datetime.strftime("%Y-%m-%d %H:%M:%S"),
#             'Matrix (mat)' : matrix,
#             'Size (int)' : Config['size'],
#             'Type (str)' : 'Stiff',
#             'Rank (int)' : -1,
#             'Density (flt)' : -1.0,
#             'Stiffness (flt)' : stiff,
#             'Other (flt)' : -1.0,
#             'Test_solver (str)' : '-',
#             'Test (flt)' : -1.0,
#             'Test_state (vec)' : [],
#             'Test_iterations (int)' : -1,
#             'Test_time (flt)' : -1.0,
#             'Train_solver (str)' : '-',
#             'Preprocessing (str)' : '-',
#             'Postprocessing (str)' : '-',
#             'Train (flt)' : -1.0,
#             'Train_state (vec)' : [],
#             'Train_iterations_1 (int)' : -1,
#             'Train_iterations_2 (int)' : -1,
#             'Train_iterations_3 (int)' : -1,
#             'Train_shots (int)': -1, 
#             'Train_cost (flt)' : -1.0,
#             'Train_time (flt)' : -1.0, 
#             'Flags (str)' : '',
#             'Baren (int)' : -1
#         }
        
#         log.log(data)

    
display(log.read(1))

{'Id (int)': 1,
 'Valid (int)': 1,
 'Updated (str)': '2025-08-21 19:18:30',
 'Matrix (mat)': array([[-5.23002864e-02, -2.68834818e-02, -3.15316187e-01,
          9.78478304e-02,  2.22056556e-02,  1.83760454e-01,
         -2.77481519e-01, -4.01682828e-01, -5.85620262e-01,
         -4.24839406e-01],
        [-2.68834818e-02, -8.77333081e-02,  3.06976163e-02,
         -1.04606479e-02, -4.60131456e-02,  2.87504271e-01,
          1.58384051e-01,  5.35786945e-02,  2.79204656e-01,
          5.79105824e-02],
        [-3.15316187e-01,  3.06976163e-02, -2.66715034e-01,
          2.94488228e-01,  9.47817798e-03,  5.54751312e-01,
         -1.11646929e-01,  1.04110309e-01,  8.61627666e-02,
         -1.84939857e-01],
        [ 9.78478304e-02, -1.04606479e-02,  2.94488228e-01,
         -1.74619894e-01,  1.83422896e-01, -3.25599342e-01,
         -5.19229246e-04,  1.81519405e-01, -7.62520100e-02,
          1.38071144e-01],
        [ 2.22056556e-02, -4.60131456e-02,  9.47817798e-03,
          1.83422896

In [21]:
import pennylane as qml 
testdev = qml.device ('default.qubit', wires = Config['size'], shots = Config['shots'])


In [22]:

def test (ids : np.ndarray, solver : str = 'bruteforce', log_data : bool = False, progressbar : bool = False ):

    ''' 
    ids : np.ndarray 
        a list of ids to compute 
    solver : str 
        solver to compute with 
    ''' 
    
    tester = Solver(dev = testdev, ansatz = '')

    if (progressbar):
        id_range = tqdm(ids, desc = f"Testing matrices with ids {min(ids)} <--> {max(ids)}") 
    else: 
        id_range = ids
    processed_data = []
    for id in id_range:

        data = log.read(id)

        print(data)
        def test_solver(Q : np.ndarray, solver : str = 'bruteforce'): 
            ''' 
            tester wrapped
            '''

            match solver: 
                case 'bruteforce':
                    J, h = prep.isingForm(Q)
                    return tester.checkIsing(J, h)
                case 'GW':
                    pass
                case 'Annealing': 
                    pass
            
        Q = data['Matrix (mat)']

        tested_data = data.copy()
        
        test, test_state, test_time = test_solver(Q)
        current_datetime = datetime.now()

        tested_data['Test (flt)'] = test 
        tested_data['Test_state (vec)'] = test_state[0].numpy()
        tested_data['Test_time (flt)'] = test_time
        tested_data['Test_solver (flt)'] = solver
        tested_data['Updated (str)'] = current_datetime.strftime("%Y-%m-%d %H:%M:%S")

        processed_data.append(tested_data)

        if (log_data): 
            log.log(tested_data, byid = True, Id = id)
    return processed_data


In [23]:
test(range(1,21), solver = 'bruteforce')

{'Id (int)': 1, 'Valid (int)': 1, 'Updated (str)': '2025-08-21 14:20:48', 'Matrix (mat)': array([[-5.23002864e-02, -2.68834818e-02, -3.15316187e-01,
         9.78478304e-02,  2.22056556e-02,  1.83760454e-01,
        -2.77481519e-01, -4.01682828e-01, -5.85620262e-01,
        -4.24839406e-01],
       [-2.68834818e-02, -8.77333081e-02,  3.06976163e-02,
        -1.04606479e-02, -4.60131456e-02,  2.87504271e-01,
         1.58384051e-01,  5.35786945e-02,  2.79204656e-01,
         5.79105824e-02],
       [-3.15316187e-01,  3.06976163e-02, -2.66715034e-01,
         2.94488228e-01,  9.47817798e-03,  5.54751312e-01,
        -1.11646929e-01,  1.04110309e-01,  8.61627666e-02,
        -1.84939857e-01],
       [ 9.78478304e-02, -1.04606479e-02,  2.94488228e-01,
        -1.74619894e-01,  1.83422896e-01, -3.25599342e-01,
        -5.19229246e-04,  1.81519405e-01, -7.62520100e-02,
         1.38071144e-01],
       [ 2.22056556e-02, -4.60131456e-02,  9.47817798e-03,
         1.83422896e-01, -1.51235505e-0

[{'Id (int)': 1,
  'Valid (int)': 1,
  'Updated (str)': '2025-08-21 21:04:45',
  'Matrix (mat)': array([[-5.23002864e-02, -2.68834818e-02, -3.15316187e-01,
           9.78478304e-02,  2.22056556e-02,  1.83760454e-01,
          -2.77481519e-01, -4.01682828e-01, -5.85620262e-01,
          -4.24839406e-01],
         [-2.68834818e-02, -8.77333081e-02,  3.06976163e-02,
          -1.04606479e-02, -4.60131456e-02,  2.87504271e-01,
           1.58384051e-01,  5.35786945e-02,  2.79204656e-01,
           5.79105824e-02],
         [-3.15316187e-01,  3.06976163e-02, -2.66715034e-01,
           2.94488228e-01,  9.47817798e-03,  5.54751312e-01,
          -1.11646929e-01,  1.04110309e-01,  8.61627666e-02,
          -1.84939857e-01],
         [ 9.78478304e-02, -1.04606479e-02,  2.94488228e-01,
          -1.74619894e-01,  1.83422896e-01, -3.25599342e-01,
          -5.19229246e-04,  1.81519405e-01, -7.62520100e-02,
           1.38071144e-01],
         [ 2.22056556e-02, -4.60131456e-02,  9.47817798e-03,


In [24]:

ids = range (1,351)
cores = 70
poolsize = len(ids)//cores

from multiprocessing import Pool
from functools import partial

if (__name__ == '__main__') and (len(ids) % poolsize == 0): 
    
    solvers = [] 

    ids = np.array(ids) 
    index = 0
    pool_ids = ids.reshape(len(ids) // poolsize, poolsize)
    display(pool_ids)
    
    args = []
    for i in range (len(pool_ids)):
        if (Config['size'] > 12):
            args.append((pool_ids[i], 'GW'))
        else: 
            args.append((pool_ids[i], 'bruteforce'))
    #print(args)
    with Pool(cores) as pool:
        processed_data = pool.starmap(test, args)


for i in range(cores):
    for data in processed_data[i]:
        log.log(data = data, byid = True, Id = data['Id (int)'])

# If the error occured you still can try to get processed_data 



array([[  1,   2,   3,   4,   5],
       [  6,   7,   8,   9,  10],
       [ 11,  12,  13,  14,  15],
       [ 16,  17,  18,  19,  20],
       [ 21,  22,  23,  24,  25],
       [ 26,  27,  28,  29,  30],
       [ 31,  32,  33,  34,  35],
       [ 36,  37,  38,  39,  40],
       [ 41,  42,  43,  44,  45],
       [ 46,  47,  48,  49,  50],
       [ 51,  52,  53,  54,  55],
       [ 56,  57,  58,  59,  60],
       [ 61,  62,  63,  64,  65],
       [ 66,  67,  68,  69,  70],
       [ 71,  72,  73,  74,  75],
       [ 76,  77,  78,  79,  80],
       [ 81,  82,  83,  84,  85],
       [ 86,  87,  88,  89,  90],
       [ 91,  92,  93,  94,  95],
       [ 96,  97,  98,  99, 100],
       [101, 102, 103, 104, 105],
       [106, 107, 108, 109, 110],
       [111, 112, 113, 114, 115],
       [116, 117, 118, 119, 120],
       [121, 122, 123, 124, 125],
       [126, 127, 128, 129, 130],
       [131, 132, 133, 134, 135],
       [136, 137, 138, 139, 140],
       [141, 142, 143, 144, 145],
       [146, 1

{'Id (int)': 31, 'Valid (int)': 1, 'Updated (str)': '2025-08-21 13:49:15', 'Matrix (mat)': array([[-0.03606144,  0.11401951, -0.17872831, -0.00085578,  0.17674299,
         0.05004521, -0.1681654 , -0.17734915,  0.04144402,  0.06129022],
       [ 0.11401951, -0.18160773,  0.02422594, -0.09343261,  0.03608712,
         0.05180088,  0.02935411,  0.12211058, -0.16441097, -0.16084343],
       [-0.17872831,  0.02422594, -0.07782319,  0.1203163 , -0.21847605,
        -0.04673892,  0.03443811, -0.15807855, -0.12965754, -0.01771626],
       [-0.00085578, -0.09343261,  0.1203163 , -0.12060253, -0.03653534,
        -0.02023592,  0.28972283,  0.29994533,  0.09436785, -0.03961164],
       [ 0.17674299,  0.03608712, -0.21847605, -0.03653534,  0.49973866,
         0.43690547, -0.03210966, -0.31225156,  0.316089  , -0.10263675],
       [ 0.05004521,  0.05180088, -0.04673892, -0.02023592,  0.43690547,
         0.11466705, -0.15900143, -0.38832233,  0.09213722,  0.04982997],
       [-0.1681654 ,  0.029

Process ForkPoolWorker-268:
Process ForkPoolWorker-280:
Process ForkPoolWorker-277:
Process ForkPoolWorker-279:
Process ForkPoolWorker-267:
Process ForkPoolWorker-262:
Process ForkPoolWorker-272:
Process ForkPoolWorker-273:
Process ForkPoolWorker-229:
Process ForkPoolWorker-227:
Process ForkPoolWorker-259:
Process ForkPoolWorker-244:
Process ForkPoolWorker-219:
Process ForkPoolWorker-260:
Process ForkPoolWorker-232:
Process ForkPoolWorker-249:
Process ForkPoolWorker-221:
Process ForkPoolWorker-250:
Process ForkPoolWorker-243:
Traceback (most recent call last):
Process ForkPoolWorker-253:
Process ForkPoolWorker-230:
Process ForkPoolWorker-266:
Process ForkPoolWorker-248:
Process ForkPoolWorker-235:
Process ForkPoolWorker-223:
Process ForkPoolWorker-228:
Process ForkPoolWorker-258:
Process ForkPoolWorker-251:
Process ForkPoolWorker-255:
Process ForkPoolWorker-237:
Process ForkPoolWorker-254:
Process ForkPoolWorker-276:
Process ForkPoolWorker-233:
Process ForkPoolWorker-238:
Process ForkP

KeyboardInterrupt: 

In [25]:

def train (data : list, trainer : Solver, stepsize: float = 0.1, depth : int = 1, log_data = False): 
    ''' 
    ids : np.ndarray 
        array of ids to train
    solver : str 
        solver with which will be trained 
    preprocessing : str 
    postprocessing : str 
    '''

    #print(f'training ids {ids}')
    #print(data)
    processed_data = []
    for line in data:
        #print(type(line))
        Q = line['Matrix (mat)']

        trained_data = line.copy()
        
        train, train_bitstring, train_state, train_iterations, train_time, baren_flag, baren_index = trainer.solve(Q = Q, depth = depth, sol = line['Test (flt)'], stepsize = stepsize, log = False)

        trained_data['Train (flt)'] = train
        trained_data['Train_state (vec)'] = train_bitstring
        for i in range(1, 4): trained_data[f'Train_iterations_{i} (int)'] = train_iterations[i - 1]
        trained_data['Train_time (flt)'] = train_time
        
        current_datetime = datetime.now()
        trained_data['Updated (str)'] = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
        
        if (baren_flag == True):
            trained_data['Flags (str)'] = f'Baren_{baren_index}'
        trained_data['Train_shots (int)'] = trainer.dev.shots.total_shots
        trained_data['Train_solver (str)'] = trainer.ansatz
        trained_data['Preprocessing (str)'] = trainer.preprocessing 
        trained_data['Postprocessing (str)'] = trainer.postprocessing
        if (log_data):
            log.log(trained_data, byid = True, Id = id)
        processed_data.append(trained_data)
    return processed_data

In [26]:
from multiprocessing import Pool
import pandas as pd 

cores = 35

solvers = [] 

depth = 3

template = pd.read_csv('data/template.csv')

for i in range (cores):
    dev = qml.device ('lightning.qubit', wires = Config['size'], shots = Config['shots'])
    solvers.append(Solver(dev = dev, ansatz = 'MA-PIA', gate = 'X', iterations_limit = Config['iterations_limit'], accepterr = Config['accepterr'], baren_check = True, baren_rerr = Config['baren_err']))


log = Logger(log_template = template, log_path = source_name.format(size = Config['size'], stiffs = Config['stiffs'], stepsize = Config['stepsize'], shots = Config['shots']))
train_dataset, _ = log._load_logfile()
#print(train_dataset)
data = [[log.read(id) for id in range (id_start, id_start + len(train_dataset) // cores)] for id_start in range (1, len(train_dataset), len(train_dataset) // cores) ]

#print(data)
if (__name__ == '__main__') and (len(train_dataset) % cores):
    
    args = [(data[i], solvers[i], Config['stepsize'], depth) for i in range (cores)]
    #print(train_dataset)
    #print(args)
    with Pool(cores) as pool:
        processed_data = pool.starmap(train, args)


for i in range(cores):
    for data in processed_data[i]:
        log.log(data = data, byid = True, Id = data['Id (int)'])




Process ForkPoolWorker-308:
Process ForkPoolWorker-291:
Process ForkPoolWorker-312:
Process ForkPoolWorker-303:
Process ForkPoolWorker-302:
Process ForkPoolWorker-289:
Process ForkPoolWorker-290:
Process ForkPoolWorker-311:
Process ForkPoolWorker-296:
Process ForkPoolWorker-300:
Process ForkPoolWorker-310:
Process ForkPoolWorker-313:
Process ForkPoolWorker-315:
Process ForkPoolWorker-306:
Process ForkPoolWorker-305:


KeyboardInterrupt: 

In [ ]:
display(processed_data[3]['Id (int)'])
display(processed_data[3]['Train (flt)'])

arr = [1,2,3,4,5,6,7,8,9,10]
arr = np.array(arr)
arr = arr.reshape(len(arr) // 2, 2)

print(arr)
train(range(1,21), trainer = solvers[0], progressbar = True)
